In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

/home/kilnaar/anaconda3/envs/ai574-pocs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-10 10:30:23.161727: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 10:30:23.192913: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-10 10:30:23.192939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-10 

# Load Model

In [5]:
MODEL_ID = "GanjinZero/biobart-v2-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast = True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, 
    device_map = "auto", torch_dtype = "auto")


# Some Medical Text

In [6]:
medical_text = """
Non-steroidal anti-inflammatory drugs are not only potent analgesics and antipyretics but also nephrotoxins, and may cause 
electrolyte disarray. In addition to the commonly expected effects, including hyperkalemia, hyponatremia, acute renal injury, 
renal cortical necrosis, and volume retention, glomerular disease with or without nephrotic syndrome or nephritis can occur as 
well including after years of seemingly safe administration. Minimal change disease, secondary membranous glomerulonephritis, 
and acute interstitial nephritis are all reported glomerular lesions seen with non-steroidal anti-inflammatory use. We report a 
patient who used non-steroidal anti-inflammatory drugs for years without diabetes, chronic kidney disease, or proteinuria; he 
then developed severe nephrotic range proteinuria with 7 g of daily urinary protein excretion. Renal biopsy showed minimal 
change nephropathy, a likely secondary membranous glomerulonephritis, and acute interstitial nephritis present simultaneously
in one biopsy. Cessation of non-steroidal anti-inflammatory drug use along with steroid treatment resulted in a moderate
improvement in renal function, though residual impairment remained. Urine heavy metal screen returned with elevated levels of
urine copper, but with normal ceruloplasmin level. Workup suggested that the elevated copper levels were due to cirrhosis from
non-alcoholic fatty liver disease. The membranous glomerulonephritis is possibly linked to non-steroidal anti-inflammatory drug
exposure, and possibly to heavy metal exposure, and is clinically and pathologically much less likely to be a primary membranous
glomerulonephritis with negative serological markers.

Keywords: Minimal change disease, podocytopathy, secondary membranous glomerulonephritis, acute interstitial nephritis, non-steroidal anti-inflammatory drugs
"""

# Inference

In [7]:
def summarize(medical_text):
    try:
        prompt = f"{medical_text.strip()}"
        inputs = tokenizer(prompt, return_tensors = "pt", max_length = 1024, truncation = True, padding = True).to(model.device)                # move tensors to same device as the model
        
        with torch.no_grad():
            generated = model.generate(**inputs,
                max_new_tokens = 40, 
                num_beams = 4, 
                length_penalty = 1.5, 
                early_stopping = False, 
                no_repeat_ngram_size = 3, 
                encoder_no_repeat_ngram_size = 3
              )

        return tokenizer.decode(generated[0], skip_special_tokens = True)
    except:
        return ""

In [8]:
summary = summarize(medical_text)
print(summary)


Non-inflammatory non-steric anti- antiinflammatory drugs are all only potent analges and antipyroretics and also nephotoxins,and may cause                     aditionalelectrolyticte dis


In [9]:
import pandas as pd
import csv
df = pd.read_csv('./mtsamples.csv')
df['transcription'] = df.transcription.astype(str)
df['description'] = df.description.astype(str)

In [10]:
df['med-biobart-summary'] = df.transcription.apply(summarize)

In [11]:
df.to_csv('mtsamples_with_biobart.csv', index = False, quoting = csv.QUOTE_NONNUMERIC)